In [1]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.9 MB/s 
     |████████████████████████████████| 120 kB 65.6 MB/s 
     |████████████████████████████████| 6.6 MB 51.3 MB/s 


In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification #, TFAutoModelForSequenceClassification # tensorflow kullanılınca bu şekilde
import torch
import torch.nn.functional as F
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments


## 1. Yöntem

In [3]:
# default model ile
classifier=pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
X_train=["We are happy to see you again","We hate with him","We hope you don't hate it"]

In [5]:

def classify_it():
  res=classifier(X_train)
  print(res)

In [7]:
classify_it()

[{'label': 'POSITIVE', 'score': 0.9998565912246704}, {'label': 'NEGATIVE', 'score': 0.9984714388847351}, {'label': 'POSITIVE', 'score': 0.831962525844574}]


# 2. Yöntem

In [8]:
model_name="distilbert-base-uncased-finetuned-sst-2-english"

In [9]:
classifier=pipeline("sentiment-analysis",model=model_name)

In [10]:
classify_it()

[{'label': 'POSITIVE', 'score': 0.9998565912246704}, {'label': 'NEGATIVE', 'score': 0.9984714388847351}, {'label': 'POSITIVE', 'score': 0.831962525844574}]


# 3. Yöntem

In [11]:
model=AutoModelForSequenceClassification.from_pretrained(model_name)

In [12]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [13]:
classifier=pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)

In [14]:
classify_it()

[{'label': 'POSITIVE', 'score': 0.9998565912246704}, {'label': 'NEGATIVE', 'score': 0.9984714388847351}, {'label': 'POSITIVE', 'score': 0.831962525844574}]


In [15]:
#bunlar aynı sonuç vermekle birlikte ikinci ve üçüncü yöntem ile özellişeltirme imkanı doğar

# 4. Yöntem

In [16]:
tokens=tokenizer.tokenize(X_train[0])
token_ids=tokenizer.convert_tokens_to_ids(tokens)
input_ids=tokenizer(X_train[0])

print(f'Tokens: {tokens}')
print(f'Token ids: {token_ids}')
print(f'Input ids: {input_ids}')


Tokens: ['we', 'are', 'happy', 'to', 'see', 'you', 'again']
Token ids: [2057, 2024, 3407, 2000, 2156, 2017, 2153]
Input ids: {'input_ids': [101, 2057, 2024, 3407, 2000, 2156, 2017, 2153, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
batch=tokenizer(X_train,padding=True,truncation=True,max_length=512,return_tensors="pt") #pt:pythorch tensor, diğer parametreler vektor uzunluklarını eşitler

print(batch)

{'input_ids': tensor([[ 101, 2057, 2024, 3407, 2000, 2156, 2017, 2153,  102,    0],
        [ 101, 2057, 5223, 2007, 2032,  102,    0,    0,    0,    0],
        [ 101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [18]:
with torch.no_grad():
  outputs=model(**batch,labels=torch.tensor([1,0,0])) #labels loss fonksiyonun hesaplanmasını sağlıyor
  print(outputs)
  predictions=F.softmax(outputs.logits,dim=1)
  print(predictions)
  labels=torch.argmax(predictions,dim=1)
  print(labels)
  labels=[model.config.id2label[label_id] for label_id in labels.tolist()]
  print(labels)


SequenceClassifierOutput(loss=tensor(0.5951), logits=tensor([[-4.2498,  4.5996],
        [ 3.5218, -2.9600],
        [-0.8004,  0.7992]]), hidden_states=None, attentions=None)
tensor([[1.4345e-04, 9.9986e-01],
        [9.9847e-01, 1.5286e-03],
        [1.6804e-01, 8.3196e-01]])
tensor([1, 0, 1])
['POSITIVE', 'NEGATIVE', 'POSITIVE']


In [19]:
#burada ilk sonuç tensor olarak, 2. sonuç tam sayı olarak, 3. sonuç ise metinsel karşılığını vermektedir.

In [20]:
#yukarıda eğittiğimiz modeli kaydediyor ve daha sonra tekrar yükleyip kullanıyoruz

In [21]:
save_directory='saved'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [22]:
tokenizer=AutoTokenizer.from_pretrained(save_directory)
model=AutoModelForSequenceClassification.from_pretrained(save_directory)

In [23]:
# bu şekilde kaydedilen modeli tekrar yüklüyoruz

# 5. Bölüm/Yöntem

Türkçe eğitilmiş modeli bulup kullanma

In [24]:
''' 
https://huggingface.co/models adresinde de filtreleme yapılarak daha önce eğitilmiş model bulunabilir. 
Bu modelin adı kopyaanarak aşağıdaki gibi kullanılır
'''

' \nhttps://huggingface.co/models adresinde de filtreleme yapılarak daha önce eğitilmiş model bulunabilir. \nBu modelin adı kopyaanarak aşağıdaki gibi kullanılır\n'

In [25]:
model_name='savasy/bert-base-turkish-sentiment-cased'

In [26]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name)

In [27]:
X_train_tr=["Çok saçma bir ürün geldi","Tam bir fiyat performans ürünü", "Bu çocuktan adam olmaz"
        ,"Eh işte idare eder bence", "Hazerfenin torunu böyle olur işte","Harika bir ürün. Herkese tavsiye ederim."]

In [28]:
batch=tokenizer(X_train_tr,padding=True,truncation=True,max_length=512,return_tensors="pt") 
#pt:pythorch tensor, diğer parametreler vektor uzunluklarını eşitler
#return_tenosrs manual ayarlamak için 'return_tensors="pt"' parametresi silinir
#batch=torch.tensor(batch['input_ids']) satırı eklenir. 
#aşağıdaki outputs=model(**batch) parametresi outputs=model(batch) şeklinde değiştirlilir.

print(batch)

{'input_ids': tensor([[    2,  6191, 14790,  1981,  1993, 31069,  1014,  3392,     3,     0,
             0],
        [    2,  2846,  1993,  2872,  5390, 31069,  5582,     3,     0,     0,
             0],
        [    2,  2011, 18756,  2165,  2851,  4035,     3,     0,     0,     0,
             0],
        [    2,  6122,  3000,  8974,  2881,  4767,     3,     0,     0,     0,
             0],
        [    2,  2430,  1972,  3305,  1973, 22553, 21411,  2589,  3000,     3,
             0],
        [    2,  5053,  1993, 31069,  1014,    18,  6664,  5452,  4980,    18,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1,

In [29]:
with torch.no_grad():
  outputs=model(**batch)
  label_ids=torch.argmax(outputs.logits,dim=1)
  labels=[model.config.id2label[label_id] for label_id in label_ids.tolist()]
  print(labels)


['negative', 'negative', 'negative', 'negative', 'positive', 'positive']


# 6. Bölüm
Kendi datasetimiz ile fine-tune yapma

1. Dataseti hazırla
2. Eğitilmiş tokinizer'ı yükle. dataset -> encoding ile çağır
3. Pytorch dataseti encodings ile build et
4. Eğitlimiş modeli yükle 
5. a) Trainer'ı yükle ve train et
   b) veya native Pytorch training pipeline kullan

In [30]:
model_name="distilbert-base-uncased" #fine tune edilmemiş model

In [31]:
#data seti hazırlama

def read_imdb_split(split_dir):
  split_dir=Path(split_dir)
  texts=[]
  labels=[]
  for label_dir in ['pos','neg']:
    for text_file in (split_dir/label_dir).iterdir():
      texts.append(text_file.read_text())
      labels.append(0 if label_dir=='neg' else 1)
  
  return texts,labels


In [32]:
!mkdir aclImdb

In [33]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2022-08-30 14:02:13--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  25.0MB/s    in 3.2s    

2022-08-30 14:02:16 (25.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [34]:
!tar -xf aclImdb_v1.tar.gz

In [35]:
train_texts,train_labels=read_imdb_split('/content/aclImdb/train')
test_texts,test_labels=read_imdb_split('/content/aclImdb/test')
train_texts,val_texts,train_labels,val_labels=train_test_split(train_texts,train_labels,test_size=.2)

In [36]:
class IMDbDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


In [37]:
tokenizer=DistilBertTokenizerFast.from_pretrained(model_name)

In [38]:
train_encodings=tokenizer(train_texts,truncation=True,padding=True)
val_encodings=tokenizer(val_texts,truncation=True,padding=True)
test_encodings=tokenizer(test_texts,truncation=True,padding=True)


In [39]:
train_dataset=IMDbDataset(train_encodings,train_labels)
val_dataset=IMDbDataset(val_encodings,val_labels)
test_dataset=IMDbDataset(test_encodings,test_labels)


In [40]:
training_args=TrainingArguments(
    output_dir='./results', #çıkış klasörü
    num_train_epochs=2,
    per_device_train_batch_size=16, #eğitim batch size
    per_device_eval_batch_size=64, #değerlendirme batch size
    warmup_steps=500, #learning rate scheduler için warm up step sayısı. check point kaç adımda bir kaydedilecek
    learning_rate=5e-5, # 
    weight_decay=0.01, # ağırlık kaybı şiddeti
    logging_dir='./logs',
    logging_steps=10,
)

In [41]:
model=DistilBertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [42]:
trainer=Trainer(
    model=model, #
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [43]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Step,Training Loss
10,0.691700
20,0.701500
30,0.687200
40,0.675500
50,0.693700
60,0.677500
70,0.657900
80,0.661700
90,0.596500
100,0.545000


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2500, training_loss=0.23061650937795639, metrics={'train_runtime': 2042.1605, 'train_samples_per_second': 19.587, 'train_steps_per_second': 1.224, 'total_flos': 5298695946240000.0, 'train_loss': 0.23061650937795639, 'epoch': 2.0})

In [ ]:
# daha sonra 3. bölümdeki gibi fine tuned model test edilebilir.

In [45]:
model=AutoModelForSequenceClassification.from_pretrained('/content/results/checkpoint-2500')
tokenizer=AutoTokenizer.from_pretrained(model_name)
classifier=pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)


loading configuration file /content/results/checkpoint-2500/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/results/checkpoint-2500",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file /content/results/checkpoint-2500/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at /cont

In [46]:
classify_it()

[{'label': 'LABEL_1', 'score': 0.9945259094238281}, {'label': 'LABEL_1', 'score': 0.9407174587249756}, {'label': 'LABEL_0', 'score': 0.6072593927383423}]


# 7. Bölüm
Native pytorch kullanarak fine tune eğitimi

In [47]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [48]:
device=torch.device('cuda')  if torch.cuda.is_available() else torch.device('cpu')

In [49]:
device

device(type='cuda')

In [50]:
model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [51]:
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)

In [52]:
optim=AdamW(model.parameters(),lr=5e-5)
num_train_epochs=2

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [53]:
for epoch in range(num_train_epochs):
  for batch in train_loader:
    optim.zero_grad()
    input_ids=batch['input_ids'].to(device)
    attention_mask=batch['attention_mask'].to(device)
    labels=batch['labels'].to(device)

    outputs=model(input_ids,attention_mask=attention_mask,labels=labels)

    loss=outputs[0]
    loss.backward()
    optim.step()

model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [57]:
 model.save_pretrained('bolum7_model')

Configuration saved in bolum7_model/config.json
Model weights saved in bolum7_model/pytorch_model.bin


In [58]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained('/content/bolum7_model')
classifier=pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

In [59]:
classify_it()

[{'label': 'LABEL_1', 'score': 0.9927115440368652}, {'label': 'LABEL_1', 'score': 0.9548053741455078}, {'label': 'LABEL_1', 'score': 0.7460792064666748}]
